# Desafio 1: Processamento de Dados de ERP de Restaurante

Autor: Marcelo Veras

Data: 25/07/2025

# Introdução

Este notebook documenta a solução para o primeiro Desafio de Engenharia de Dados do Laboratório de Tecnologia do Coco Bambu (CB Lab). O objetivo é processar um arquivo JSON de um ERP, modelar os dados para um banco de dados relacional e justificar a abordagem de design escolhida.

## 1. Descrição do Esquema JSON (ERP.json)

### 1.1 O JSON fornecido possui um esquema hierárquico que representa transações de um ponto de venda. A estrutura principal é a seguinte:

#### Objeto Raiz: Contém metadados sobre a carga de dados.
* curUTC (String): Timestamp em UTC de quando o dado foi gerado.

* locRef (String): Uma referência ou ID da loja/localidade. "99 CB CB".

* guestChecks (Array de Objetos): Uma lista contendo um ou mais pedidos.

#### Objeto guestCheck (Pedido): Cada objeto na lista guestChecks representa um único pedido e contém:

* Identificadores: guestCheckId (Number), chkNum (Number), empNum (Number), rvcNum (Number), etc.

* Timestamps: Diversos campos de data/hora para abertura, fechamento e atualizações (opnBusDt, opnUTC, clsdUTC, etc.).

* Valores Monetários: subTtl (Number), chkTtl (Number), dscTtl (Number, pode ser negativo), payTtl (Number).

* Flags e Contadores: clsdFlag (Boolean), gstCnt (Number), numSrvcRd (Number).

* taxes (Array de Objetos): Uma lista dos impostos aplicados ao pedido. Cada objeto de imposto contém taxNum, txblSlsTtl, taxCollTtl, etc.

* detailLines (Array de Objetos): O coração do pedido, representando cada linha de item individual.

#### Objeto detailLine (Linha de Detalhe): Cada objeto na lista detailLines representa um evento ou item no pedido.

* Identificadores e Metadados: guestCheckLineItemId (Number), lineNum (Number), detailUTC (String), etc.

* Valores: dspTtl (Number), dspQty (Number).

* Objeto de Conteúdo Específico: Este é o campo polimórfico. No exemplo, ele contém:

* MenuItem (Objeto): Detalhes de um item do cardápio, como miNum (ID do item), modFlag (se foi modificado), inclTax (imposto incluso), etc.

* Conforme o enunciado, aqui também poderiam estar objetos como discount, serviceCharge, tenderMedia ou errorCode.

### 1.2 Carregamento e Análise do Esquema JSON

Nesta seção, o arquivo ERP.json é carregado e sua estrutura é explorada para entender o esquema dos dados.

In [3]:
import json
import pandas as pd

file_path = '../data/ERP.json'

with open(file_path, 'r') as f:
    data = json.load(f)

print("Estrutura principal do JSON:")
print(data.keys())

print("\nEstrutura de 'guestChecks':")
guest_check_exemplo = data['guestChecks'][0]
print(guest_check_exemplo.keys())

print("\nEstrutura de 'detailLines':")
detail_line_exemplo = guest_check_exemplo['detailLines'][0]
print(detail_line_exemplo.keys())



Estrutura principal do JSON:
dict_keys(['curUTC', 'locRef', 'guestChecks'])

Estrutura de 'guestChecks':
dict_keys(['guestCheckId', 'chkNum', 'opnBusDt', 'opnUTC', 'opnLcl', 'clsdBusDt', 'clsdUTC', 'clsdLcl', 'lastTransUTC', 'lastTransLcl', 'lastUpdatedUTC', 'lastUpdatedLcl', 'clsdFlag', 'gstCnt', 'subTtl', 'nonTxblSlsTtl', 'chkTtl', 'dscTtl', 'payTtl', 'balDueTtl', 'rvcNum', 'otNum', 'ocNum', 'tblNum', 'tblName', 'empNum', 'numSrvcRd', 'numChkPrntd', 'taxes', 'detailLines'])

Estrutura de 'detailLines':
dict_keys(['guestCheckLineItemId', 'rvcNum', 'dtlOtNum', 'dtlOcNum', 'lineNum', 'dtlId', 'detailUTC', 'detailLcl', 'lastUpdateUTC', 'lastUpdateLcl', 'busDt', 'wsNum', 'dspTtl', 'dspQty', 'aggTtl', 'aggQty', 'chkEmpId', 'chkEmpNum', 'svcRndNum', 'seatNum', 'menuItem'])


######

##### Com base na exploração acima, o esquema foi documentado detalhadamente na resposta do desafio. A análise confirma a natureza hierárquica e a presença de listas aninhadas (guestChecks, taxes, detailLines), sendo detailLines um campo polimórfico.